In [4]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import copy
from sklearn.metrics import f1_score

from utils import config
from utils import utils
from utils.models import Regressor, FeatureExtractor, ActivityClassifier

if config.dataset == config.Dataset.MMFIT:
    import utils.mmfit_data as mmfit
    window_length = config.sensor_window_length
elif config.dataset == config.Dataset.MHAD:
    import utils.mhad_data as mhad
    window_length = config.mhad_window_length

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# # >>> REGRESSOR <<< #
pose2imu_model = Regressor(
    in_ch=config.in_ch, num_joints=config.num_joints, window_length=window_length
).to(config.device)
best_seed = config.best_pose2imu_seed # best_seed of best model
model_name = config.pose2imu_model_name + f"[{best_seed}](model).pth"
latest_model = utils.find_latest_model(model_name)
print(f"loading: {latest_model}")
pose2imu_model.load_state_dict(torch.load(latest_model, map_location=config.device))
pose2imu_model.eval() # we're not training this model

fe_model = FeatureExtractor().to(config.device, non_blocking=True)
ac_model = ActivityClassifier(f_in=config.ac_fin, n_classes=config.ac_num_classes).to(config.device, non_blocking=True)

criterion = nn.CrossEntropyLoss()
params = list(fe_model.parameters()) + list(ac_model.parameters())
optimizer = torch.optim.Adam(params=params, lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=5, verbose=True
)

loading: ../train_out/22.02/16.12_pose2imu[1000000](model).pth


In [4]:
# # --- TEST f_in
# fe_model.eval()
# with torch.no_grad():
#     for pose, acc, labels in mmfit.train_loader:
#         out = fe_model(acc)
#         print(out.shape)
#         out = out.view(out.shape[0], -1)
#         print(out.shape)
#         out = ac_model(out)
#         print(out.shape)
#         break

In [5]:
# >>> Training <<< #
train_loss_history, train_f1_history, train_accuracy_history = [], [], []
val_loss_history, val_f1_history, val_accuracy_history = [], [], []

best_fe_model_state, best_ac_model_state = None, None
best_epoch = float('inf')
best_val_f1 = 0
best_val_acc = 0

epochs = config.epochs
patience = config.patience
epochs_no_improve = 0

log = ''

for epoch in range(epochs):

    # - TRAIN
    total_train_loss = 0
    all_pred_labels, all_true_labels = [], []
    total_predictions, correct_predictions = 0, 0

    fe_model.train()
    ac_model.train()
    for pose, acc, labels in mmfit.train_loader:
        """
        pose: (batch_size, 3, num_joints, sensor_window_length)
        acc: (batch_size, 3, sensor_window_length)
        """
        # -- Move to GPU
        pose = pose.to(config.device, non_blocking=True)
        acc = acc.to(config.device, non_blocking=True)
        labels = labels.to(config.device, non_blocking=True)

        # -- Forward pass
        # --- Regressor
        if config.mode == config.Mode.SIM:
            acc = pose2imu_model(pose)  # IMPORTANT SIM acc
        # --- Feature Extractor
        acc_features = fe_model(acc)
        # --- Activity Classifier
        label_logits = ac_model(acc_features)

        # -- Loss
        loss = criterion(label_logits, labels)
        total_train_loss += loss.item()

        # -- F1
        pred_labels = torch.argmax(label_logits, dim=1)
        all_pred_labels.extend(pred_labels.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

        # -- Accuracy
        total_predictions += labels.size(0)
        correct_predictions += (pred_labels == labels).sum().item()

        # -- Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # -- Loss
    average_train_loss = total_train_loss / len(mmfit.train_loader)
    train_loss_history.append(average_train_loss)

    # -- F1
    train_f1 = f1_score(all_true_labels, all_pred_labels, average="macro")
    train_f1_history.append(train_f1)

    # -- Accuracy
    train_accuracy = correct_predictions / total_predictions
    train_accuracy_history.append(train_accuracy)

    # ------------------------------------------------------------------------------------------------------- #

    # - VAL
    total_val_loss = 0
    all_pred_labels, all_true_labels = [], []
    total_predictions, correct_predictions = 0, 0

    fe_model.eval()
    ac_model.eval()
    with torch.no_grad():
        for pose, acc, labels in mmfit.val_loader:
            # -- Move to GPU
            pose = pose.to(config.device, non_blocking=True)
            acc = acc.to(config.device, non_blocking=True)
            labels = labels.to(config.device, non_blocking=True)

            # -- Forward pass
            # --- Feature Extractor
            acc_features = fe_model(acc)
            # --- Activity Classifier
            label_logits = ac_model(acc_features)

            # -- Loss
            loss = criterion(label_logits, labels)
            total_val_loss += loss.item()

            # -- F1
            pred_labels = torch.argmax(label_logits, dim=1)
            all_pred_labels.extend(pred_labels.cpu().numpy())
            all_true_labels.extend(labels.cpu().numpy())

            # -- Accuracy
            total_predictions += labels.size(0)
            correct_predictions += (pred_labels == labels).sum().item()

    # -- Loss
    average_val_loss = total_val_loss / len(mmfit.val_loader)
    val_loss_history.append(average_val_loss)

    # -- F1
    val_f1 = f1_score(all_true_labels, all_pred_labels, average="macro")
    val_f1_history.append(val_f1)

    # -- Accuracy
    val_accuracy = correct_predictions / total_predictions
    val_accuracy_history.append(val_accuracy)

    # ------------------------------------------------------------------------------------------------------- #

    out = (f"Epoch {epoch+1}/{epochs}" +
           f"\nTRAIN Total Loss: {average_train_loss:.4f}" +
           f'\nTRAIN F1: {train_f1:.4f}, Accuracy: {train_accuracy:.4f}' +
           f'\nVAL Total Loss: {average_val_loss:.4f}' +
           f'\nVAL F1: {val_f1:.4f}, Accuracy: {val_accuracy:.4f}' +
           f'\n----------------------------------------------------\n')

    print(out)

    # - Early stop
    if best_val_f1 < val_f1:
        epochs_no_improve = 0

        best_val_f1 = val_f1
        best_val_acc = val_accuracy
        best_epoch = epoch
        
        best_fe_model_state = copy.deepcopy(fe_model.state_dict())
        best_ac_model_state = copy.deepcopy(ac_model.state_dict())
        
        log = out

    else:
        epochs_no_improve += 1

    # Training early stop
    if epochs_no_improve == patience:
        # -- Load best model
        fe_model.load_state_dict(best_fe_model_state)
        ac_model.load_state_dict(best_ac_model_state)

        break

    scheduler.step(average_val_loss)

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
ERROR:tornado.general:SEND Error: Host unreachable
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


KeyboardInterrupt: 

In [5]:
# >>> Test <<< #
# - Test
total_loss = 0
loss_history, f1_history, accuracy_history = [], [], []
all_pred_labels, all_true_labels = [], []
total_predictions, correct_predictions = 0, 0

fe_model.eval()
ac_model.eval()
with torch.no_grad():
    for pose, acc, labels in mmfit.test_loader:
        # -- Move to GPU
        acc = acc.to(config.device, non_blocking=True)
        labels = labels.to(config.device, non_blocking=True)

        # -- Forward pass
        # --- Feature Extractor
        acc_features = fe_model(acc)
        # --- Activity Classifier
        label_logits = ac_model(acc_features)

        # -- Loss
        loss = criterion(label_logits, labels)
        total_loss += loss.item()

        # -- F1 
        pred_labels = torch.argmax(label_logits, dim=1)
        all_pred_labels.extend(pred_labels.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

        # -- Accuracy
        total_predictions += labels.size(0)
        correct_predictions += (pred_labels == labels).sum().item()

# -- Loss
average_loss = total_loss / len(mmfit.test_loader)
loss_history.append(average_loss)

# -- F1
f1 = f1_score(all_true_labels, all_pred_labels, average="macro")

# -- Accuracy
accuracy = correct_predictions / total_predictions

log += f"Test F1: {f1:.4f}, Test Accuracy: {accuracy:.4f}" + \
        f'\n----------------------------------------------------\n'

In [ ]:
# >>> Save models and metrics <<< #
if config.mode == config.Mode.REAL:
    mode = "real"
elif config.mode == config.Mode.SIM:
    mode = "sim"
elif config.mode == config.Mode.COMB:
    mode = "comb"

prefix = config.scenario1_name + "[s=" + str(utils.args.seed) + "]" + "[m=" + mode + "]"

# - Models
# file_name = config.ae_model_name + prefix + '(model)'
# utils.save_model(best_ae_model_state, file_name)

# file_name = config.fc_model_name + prefix + '(model)'
# utils.save_model(best_fc_model_state, file_name)

# - Loss
metric = "Total Loss"
file_name = '1_' + prefix + "(" + metric + ")"
utils.save_plot(
    epochs=epoch,
    best_epoch=best_epoch,
    train_metric_history=train_loss_history,
    val_metric_history=val_loss_history,
    metric=metric,
    file_name=file_name,
)

# - F1
metric = "F1"
file_name = '2_' + prefix + "(" + metric + ")"
utils.save_plot(
    epochs=epoch,
    best_epoch=best_epoch,
    train_metric_history=train_f1_history,
    val_metric_history=val_f1_history,
    metric=metric,
    file_name=file_name,
)

# - Accuracy
metric = "Accuracy"
file_name = '3_' + prefix + "(" + metric + ")"
utils.save_plot(
    epochs=epoch,
    best_epoch=best_epoch,
    train_metric_history=train_accuracy_history,
    val_metric_history=val_accuracy_history,
    metric=metric,
    file_name=file_name,
)

# - Log
log += prefix
file_name = '4_' + prefix + "(Log)"
utils.save_log(log=log, file_name=file_name)